In [ ]:
# Transformer Model From Scratch

In [ ]:
Install required datasets, including accelerate library for doing the training

In [6]:
!pip install transformers datasets
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 76.3 MB/s eta 0:00:00
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 51.9 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [ ]:
Load the datasets into memory

In [8]:
from datasets import load_dataset
dataset = load_dataset('jeffnyman/emotions')

README.md:   0%|          | 0.00/5.80k [00:00<?, ?B/s]

emotions.py:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

The repository for jeffnyman/emotions contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/jeffnyman/emotions.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


ModuleNotFoundError: No module named 'datasets.tasks'

In [ ]:
Define the tokenizer function/strategy, based on the Bert-Base-Uncased

In [ ]:
from transformers import AutoTokenizer

def tokenize_function(examples):
  return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
Verify unique labels in the dataset to prevent GPU-related errors

In [ ]:
unique_labels = set(tokenized_datasets['train']['label'])
print(f"Unique labels in the training set: {unique_labels}")

def check_labels(dataset):
  for label in dataset['train']['label']:
    if label not in unique_labels:
      print(f"Found invalid label: {label}")

check_labels(tokenized_datasets)

In [ ]:
Specify Hyperparameters and creates the Model Config

In [ ]:
from transformers import BertConfig
from transformers import BertForSequenceClassification

config = BertConfig(
vocab_size=tokenizer.vocab_size,
hidden_size=512,
num_hidden_layers=6,
num_attention_heads=8,
intermediate_size=2048,
max_position_embeddings=512,
num_labels=len(unique_labels)
)

model = BertForSequenceClassification(config)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  output_dir='./results',
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=3,
  weight_decay=0.01,
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["test"],
)

In [ ]:
Now run the train function for this model

In [ ]:
trainer.train()